In [ ]:
# !pip install transformers==4.57.1

# 5.1 - Introdução ao BERT: Pré-treinamento e Arquitetura

Na aula anterior, exploramos detalhadamente o **Transformer**, arquitetura proposta no artigo *Attention is All You Need (Vaswani et al., 2017)*.  
Vimos como o modelo substitui as recorrências por **atenção auto-regressiva**, resultando em paralelização e aprendizado mais eficiente de dependências de longo alcance.

O **BERT (Bidirectional Encoder Representations from Transformers)**, proposto por **Devlin et al. (2018)**, é um passo além dessa ideia.  
Ele se baseia **exclusivamente na parte do encoder do Transformer**, mas introduz um novo paradigma: **aprendizado de representações de linguagem profundas e bidirecionais**, usando **pré-treinamento auto-supervisionado**.

<br/>
<img src="https://raw.githubusercontent.com/bmnogueira-ufms/TopicosIA-2025-02/main/images/bert_pt_ft.png" width="50%%">

Fonte: Devlin et al. (2018)

---

## Por que BERT?

Antes do BERT, os modelos de linguagem típicos (como GPT-1 ou ELMo) processavam texto **em apenas uma direção**:

- Modelos *left-to-right* (como GPT): preveem o próximo token a partir dos anteriores.  
- Modelos *right-to-left* (como LM reversos): preveem o token anterior.  
- ELMo (2018) combinava duas redes LSTM unidirecionais, mas de forma ainda limitada.

Essas abordagens **não capturavam o contexto completo** de uma palavra em sua sentença, já que a representação dependia apenas do passado ou do futuro, mas não dos dois **ao mesmo tempo**.

O BERT resolve isso ao:
- Treinar **bidirecionalmente**, olhando para a **sentença inteira**;  
- Usar um esquema de pré-treinamento **auto-supervisionado** (sem rótulos humanos);  
- E depois **ajustar** o modelo (*fine-tuning*) para tarefas específicas de NLP.

---

## Transfer Learning em NLP

Assim como o *ImageNet* revolucionou a visão computacional com o conceito de **pré-treinamento + fine-tuning**, o BERT trouxe essa revolução para o **Processamento de Linguagem Natural (PLN)**.

O processo segue duas fases:

1. **Pré-treinamento (Pretraining)**  
   - O modelo aprende conhecimento linguístico geral a partir de **grandes corpora** (Wikipedia + BooksCorpus).  
   - Duas tarefas auto-supervisionadas são usadas:
     - **Masked Language Modeling (MLM)**: prever palavras mascaradas no texto.  
     - **Next Sentence Prediction (NSP)**: prever se uma sentença B segue naturalmente uma sentença A.

2. **Ajuste fino (Fine-tuning)**  
   - O modelo é adaptado para uma tarefa específica: classificação, QA, NER, etc.  
   - Os mesmos pesos são reutilizados, e apenas algumas camadas finais são ajustadas.

Essa combinação tornou o BERT um **modelo universal de linguagem**, facilmente adaptável a diversas tarefas de NLP.

---

## Intuição: O que o BERT aprende?

Durante o pré-treinamento, o BERT **aprende as relações contextuais** entre as palavras:
- Como palavras diferentes se associam em frases;  
- Que estrutura sintática e semântica definem o sentido;  
- Que palavras são prováveis em determinados contextos.

Com isso, ele gera **embeddings contextuais**, ou seja, o vetor de uma palavra depende da **sentença em que ela aparece**.

> Exemplo:  
> - Em “Ele foi ao **banco** sacar dinheiro”, “banco” → instituição financeira.  
> - Em “Ela sentou no **banco** do parque”, “banco” → assento.  
>
> O BERT gera embeddings diferentes para cada uso.

---

## Arquitetura Geral

<br/>
<img src="https://raw.githubusercontent.com/bmnogueira-ufms/TopicosIA-2025-02/main/images/bert_size_architecture.png" width="50%%">

Fonte: [Huggingface](https://huggingface.co/blog/bert-101)

O BERT baseia-se **no encoder do Transformer**, repetido *N* vezes.  
Cada camada contém:

1. **Multi-Head Self-Attention**  
   - Permite que cada token atenda a todos os outros tokens da sentença, em ambas as direções.  
   - É aqui que o “Bidirectional” de BERT acontece.

2. **Feed-Forward Network (FFN)**  
   - Aplica transformações não lineares em cada posição, aprendendo representações mais ricas.

3. **Residual Connections + Layer Normalization**  
   - Facilitam o fluxo de gradientes e a estabilidade do treinamento.

---

## Configurações originais

| Modelo | Camadas (L) | Heads | Dimensão (`d_model`) | Dim. Feedforward | Parâmetros |
|:--------|:------------:|:------:|:--------------------:|:----------------:|:-----------:|
| **BERT Base** | 12 | 12 | 768 | 3072 | 110M |
| **BERT Large** | 24 | 16 | 1024 | 4096 | 340M |

---

## Entrada do BERT

O BERT processa o texto em forma de tokens especiais:

- **[CLS]** → marca o início da sequência (usado para classificação).  
- **[SEP]** → separa sentenças (usado em tarefas de pares).  
- **[MASK]** → substitui tokens que o modelo deve prever.

> Exemplo:  
> Entrada: `[CLS] O gato [MASK] no tapete [SEP]`  
> Saída: prever a palavra “dormiu”.

# 5.2 - Objetivos de Pré-Treinamento do BERT

O sucesso do BERT vem de seu **pré-treinamento auto-supervisionado** em larga escala.  
Ele não usa rótulos humanos: aprende diretamente com **texto cru**, aplicando duas tarefas artificiais que forçam o modelo a entender a estrutura e o sentido da linguagem.

Essas duas tarefas são:

1. **Masked Language Modeling (MLM)**  
2. **Next Sentence Prediction (NSP)**

Vamos entender cada uma delas com muito detalhe.

---

## 5.2.1. Masked Language Modeling (MLM)

O objetivo do **MLM** é ensinar o modelo a *prever palavras faltantes* (ou mascaradas) a partir do contexto.

### Intuição

Imagine a seguinte frase:

> “O cachorro **[MASK]** o carteiro.”

O modelo deve prever a palavra mais provável para **[MASK]**, considerando **todas as palavras** da sentença — tanto antes quanto depois.

Dessa forma, o BERT aprende **representações bidirecionais**, já que olha para a esquerda e para a direita ao mesmo tempo.

---

### Como o processo funciona

1. **Tokenização e inserção de tokens especiais**  
   O texto é dividido em *subtokens* (usando WordPiece) e recebe marcadores especiais:

> [CLS] O cachorro [MASK] o carteiro [SEP]


2. **Escolha aleatória dos tokens a mascarar**  
Em cada sequência, **15% dos tokens** são selecionados para mascaramento.

3. **Substituições aplicadas (segundo o paper original):**
- 80% → substituídos por `[MASK]`  
  > Ex: “O cachorro **[MASK]** o carteiro”
- 10% → substituídos por uma palavra **aleatória**  
  > Ex: “O cachorro **comeu** o carteiro”  
- 10% → **mantidos iguais**, mas ainda contam como alvos de predição  
  > Ex: “O cachorro **mordeu** o carteiro”

Isso força o modelo a não se apoiar apenas no token `[MASK]` e a construir **representações contextuais robustas**.

---

### Exemplo Prático

Entrada:

> [CLS] O cachorro [MASK] o carteiro [SEP]

Saída esperada:

> [CLS] O cachorro mordeu o carteiro [SEP]

Durante o treinamento, o modelo gera uma distribuição de probabilidade sobre todo o vocabulário e tenta **maximizar a probabilidade do token correto**.

---

### Resultado

Ao final do pré-treino, o BERT aprende vetores de embeddings altamente contextuais:
- A palavra “mordeu” está associada a “cachorro” e “carteiro”.
- Se a mesma palavra aparecer em outro contexto (“Ele **mordeu** a língua”), o vetor muda, refletindo o novo sentido.

---

## 5.2.2. Next Sentence Prediction (NSP)

Além de entender palavras, o BERT também precisa compreender **relações entre sentenças**.  
Essa é a função do **Next Sentence Prediction (NSP)**.

---

### Intuição

O NSP ensina o modelo a entender **coerência e continuidade textual**.

Ele recebe **dois segmentos de texto (A e B)** e precisa responder:

> “A sentença B realmente vem depois da sentença A no texto original?”

---

### Como o processo funciona

Durante o pré-treinamento:

- 50% dos pares (A,B) são **verdadeiros** → B realmente segue A.  
- 50% dos pares são **falsos** → B vem de uma parte aleatória do corpus.

O modelo deve classificar entre:

| Rótulo | Significado | Exemplo |
|:-------|:-------------|:---------|
| `IsNext` | B segue A | A: “Ele pegou o guarda-chuva.”<br>B: “Saiu para a rua.” |
| `NotNext` | B é aleatória | A: “Ele pegou o guarda-chuva.”<br>B: “O bolo estava delicioso.” |

---

### Entrada típica do BERT para NSP

O modelo recebe os dois segmentos concatenados, separados por tokens especiais:

> [CLS] Ele pegou o guarda-chuva. [SEP] Saiu para a rua. [SEP]

Além disso, ele adiciona **embeddings segmentais** (também chamados *token type embeddings*), que indicam se cada token pertence à sentença A ou B.

| Tipo de embedding | Função |
|:------------------|:--------|
| **Token embeddings** | Representa cada palavra ou subtoken. |
| **Segment embeddings** | Indicam a qual sentença o token pertence (A ou B). |
| **Positional embeddings** | Indicam a posição de cada token na sequência. |

O vetor final de entrada de cada token é a **soma** desses três componentes.

---

### Exemplo Prático

Entrada:

> [CLS] O cachorro latiu. [SEP] O carteiro correu. [SEP]

Rótulo: `IsNext`

Durante o pré-treino, o BERT aprende:
- Que “latiu” e “carteiro” ocorrem juntos com frequência;  
- Que há relação causal ou temporal entre as sentenças.

Isso o prepara para tarefas posteriores como *Question Answering*, *Natural Language Inference* e *Paraphrase Detection*.

---

## Saída do Pré-Treinamento

Durante o pré-treino, o BERT otimiza **duas funções de perda simultaneamente**:

$$
\mathcal{L} = \mathcal{L}_{MLM} + \mathcal{L}_{NSP}
$$

- **$\mathcal{L}_{MLM}$** → erro de predição das palavras mascaradas.  
- **$\mathcal{L}_{NSP}$** → erro de classificação entre `IsNext` e `NotNext`.

A combinação dessas perdas permite que o modelo aprenda tanto **relações locais (entre palavras)** quanto **globais (entre sentenças)**.

---

## Resumo Visual

O BERT é, portanto, um **modelo de linguagem bidirecional e contextual**, treinado de forma auto-supervisionada, capaz de ser ajustado para praticamente qualquer tarefa de PLN.


<pre>
Entrada:
[CLS] A sentença A ... [SEP] sentença B ... [SEP]

↓ Embeddings = Token + Segment + Position

↓ Encoder do Transformer (12 camadas)

↓
Saídas:
 - Vetores de cada token (para MLM)
 - Vetor [CLS] (para NSP)
</pre>

# 5.3 Os três tipos de embeddings no BERT

<br/>
<img src="https://raw.githubusercontent.com/bmnogueira-ufms/TopicosIA-2025-02/main/images/bert_embeddings.png" width="50%%">

Fonte: Devlin et al. (2018)

O BERT não usa apenas *word embeddings*.  
Cada token de entrada é representado pela **soma de três vetores** diferentes:

$$
\text{InputEmbedding}(t_i) = \text{TokenEmb}(t_i) + \text{SegmentEmb}(s_i) + \text{PositionEmb}(p_i)
$$

---

## 5.3.1 Token Embeddings
São os embeddings “normais” de palavras, como nos modelos anteriores (Word2Vec, GloVe, etc).  
Cada palavra (ou subpalavra, no caso do BERT) é convertida em um vetor denso.

Exemplo:
```text
[CLS] o filme foi ótimo [SEP]
```

Os tokens podem ser:
```text
[CLS], o, fil, ##me, foi, ót, ##imo, [SEP]
```

Cada um recebe um vetor aprendido — esses vetores formam a **base semântica** da frase.

---

## 5.3.2 Position Embeddings
Transformers não têm noção de ordem (eles não são recorrentes).  
Por isso, o BERT adiciona **positional embeddings fixos** que indicam a posição de cada token na sequência.

Esses vetores são somados ao embedding de cada token e permitem que o modelo diferencie, por exemplo:

```text
o cachorro mordeu o homem
o homem mordeu o cachorro
```

Apesar das mesmas palavras, as posições diferentes mudam completamente o significado.

---

## 5.3.3 Segment (ou Sentence) Embeddings

Durante o pré-treinamento, o BERT usa o **Next Sentence Prediction (NSP)**, que exige lidar com **pares de sentenças**.  
Para que o modelo saiba **qual token pertence a qual sentença**, o BERT adiciona embeddings de segmento:

| Token | Segment Embedding |
|:-------|:-----------------:|
| `[CLS]` | A |
| `O` | A |
| `filme` | A |
| `foi` | A |
| `ótimo` | A |
| `[SEP]` | A |
| `Não` | B |
| `gostei` | B |
| `.` | B |
| `[SEP]` | B |

→ Tokens da **primeira sentença** recebem `Segment A`  
→ Tokens da **segunda sentença** recebem `Segment B`

Esses vetores de segmento são aprendidos junto com o resto da rede.

---

## 5.3.4 Combinação Final
Para cada token, o vetor final de entrada é a soma:

$$
\text{EmbeddingFinal}(t_i) =
\text{TokenEmb}(t_i)
+ \text{SegmentEmb}(s_i)
+ \text{PositionEmb}(p_i)
$$


Essa combinação fornece ao modelo:

- **significado** da palavra (Token),
- **ordem** na sequência (Position),
- **identificação** da sentença (Segment).

Essa soma é o **ponto de partida do aprendizado contextual** no BERT.  
Os três tipos de embeddings são aprendidos (ou fixos, no caso posicional) e evoluem durante o pré-treinamento.

---

## 5.4 — BERT na prática: Masked Language Modeling (MLM) + Next Sentence Prediction (NSP)

Nesta seção vamos **reproduzir em pequena escala** os objetivos de pré-treinamento do BERT usando `BertForPreTraining` (que já inclui as duas cabeças: **MLM** e **NSP**).

### O que vamos fazer

1. **Mini-corpus**: definimos algumas sentenças curtas (você pode substituir pelo seu conjunto).
2. **Pares para NSP**: criamos pares (A,B) rotulados:
   - `IsNext` quando B realmente segue A no texto original;
   - `NotNext` quando B é uma sentença aleatória.
3. **Tokenização**: `[CLS] A [SEP] B [SEP]`, com `token_type_ids` (A=0, B=1).
4. **Máscaras para MLM (80/10/10)** sobre ~15% dos tokens “previstáveis”.
5. **Treino curto**: otimizamos **uma única loss conjunta**:  
   $\mathcal{L} = \mathcal{L}_{MLM} + \mathcal{L}_{NSP}$
6. **Inspeção de saídas**:
   - Top-k predições para `[MASK]`;
   - Probabilidades `IsNext/NotNext` do NSP para um par de teste.

### Dicas

- **MLM** força o BERT a aprender **contexto bidirecional** (olhando esquerda e direita).  
- **NSP** ensina **coerência entre sentenças** (embora trabalhos posteriores mostrem que remover NSP às vezes ajuda; aqui mantemos por fidelidade ao BERT original).  
- Em produção, corpora são massivos (Wikipedia + BooksCorpus). Aqui, **mini-setup** para visualização.

In [ ]:
# ===========================================
# BERT Pretraining Demo: MLM + NSP (didático)
# ===========================================
import math, random, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import (
    BertTokenizerFast,
    BertForPreTraining,
)

# ---------------------------
# 0) Setup
# ---------------------------
SEED = 7
random.seed(SEED); torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------------------
# 1) Mini-corpus (edite à vontade)
#    Cada "documento" é uma lista de sentenças em ordem.
# ---------------------------
documents = [
    [
        "the cat sat on the mat.",
        "it looked out the window.",
        "then it jumped down.",
        "the dog barked loudly."
    ],
    [
        "deep learning changed natural language processing.",
        "transformers enable parallel training.",
        "bert learns bidirectional context.",
        "masked language modeling is powerful."
    ],
    [
        "paris is the capital of france.",
        "the eiffel tower is in paris.",
        "tourists visit the louvre.",
        "french cuisine is famous."
    ],
]

# ---------------------------
# 2) Gerar pares (A,B) para NSP
#    - 50% IsNext (B segue A no mesmo doc)
#    - 50% NotNext (B aleatória de outro lugar)
# ---------------------------
def build_nsp_pairs(docs, num_pairs=300):
    pairs = []
    for _ in range(num_pairs):
        doc = random.choice(docs)
        if len(doc) >= 2 and random.random() < 0.5:
            # IsNext: escolher A e B consecutivos no mesmo doc
            i = random.randint(0, len(doc) - 2)
            A, B = doc[i], doc[i+1]
            label = 0  # IsNext
        else:
            # NotNext: A de um doc e B aleatória de outro doc
            docA = random.choice(docs)
            A = random.choice(docA)
            # garanta B de outro doc para ser (em geral) NotNext
            docB = random.choice([d for d in docs if d is not docA])
            B = random.choice(docB)
            label = 1  # NotNext
        pairs.append((A, B, label))
    return pairs

pairs = build_nsp_pairs(documents, num_pairs=400)

# ---------------------------
# 3) Tokenizer e encoding de pares
# ---------------------------
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

MAX_LEN = 64
def encode_pair(a, b):
    enc = tokenizer(
        a, b,
        add_special_tokens=True,
        max_length=MAX_LEN,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    # enc: input_ids, token_type_ids, attention_mask
    return {k: v.squeeze(0) for k, v in enc.items()}

# ---------------------------
# 4) MLM: selecionar posições a mascarar (15%) e aplicar 80/10/10
#    - ignorar especiais [CLS]/[SEP]/PAD na seleção
# ---------------------------
CLS_ID = tokenizer.cls_token_id
SEP_ID = tokenizer.sep_token_id
PAD_ID = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
MASK_ID = tokenizer.mask_token_id

def apply_mlm_masking(input_ids, mlm_prob=0.15):
    labels = input_ids.clone()

    # candidatos "previstáveis" (não especiais)
    special_ids = {CLS_ID, SEP_ID, PAD_ID}
    can_mask = torch.ones_like(input_ids, dtype=torch.bool)
    for sid in special_ids:
        can_mask &= (input_ids != sid)

    # amostra binária de quais posições mascarar
    probs = torch.full(input_ids.shape, mlm_prob, device=input_ids.device)
    mask_positions = (torch.bernoulli(probs).bool()) & can_mask

    # 80% -> [MASK]
    mask80 = mask_positions & (torch.rand_like(input_ids, dtype=torch.float) < 0.8)
    input_ids[mask80] = MASK_ID

    # 10% -> token aleatório
    # (onde foi selecionado p/ máscara mas não entrou no 80%)
    remaining = mask_positions & (~mask80)
    rand10 = remaining & (torch.rand_like(input_ids, dtype=torch.float) < 0.5)
    vocab_size = tokenizer.vocab_size
    random_tokens = torch.randint(low=0, high=vocab_size, size=input_ids.shape, device=input_ids.device)
    input_ids[rand10] = random_tokens[rand10]

    # 10% -> deixam igual (implicitly: remaining & ~rand10)

    # posições não-mascaradas não entram no loss (=-100)
    labels[~mask_positions] = -100
    return input_ids, labels, mask_positions

# ---------------------------
# 5) Dataset + DataLoader
# ---------------------------
class BertPretrainDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        a, b, nsp_label = self.pairs[idx]
        enc = encode_pair(a, b)
        # aplica MLM no input_ids
        ids, labels, mask_pos = apply_mlm_masking(enc["input_ids"].clone())
        item = {
            "input_ids": ids,
            "token_type_ids": enc["token_type_ids"],
            "attention_mask": enc["attention_mask"],
            "labels": labels,  # para MLM
            "next_sentence_label": torch.tensor(nsp_label, dtype=torch.long)
        }
        return item

dataset = BertPretrainDataset(pairs)
split = int(0.9 * len(dataset))
train_ds, val_ds = torch.utils.data.random_split(dataset, [split, len(dataset)-split])

BATCH = 8
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False)

# ---------------------------
# 6) Modelo (MLM + NSP) e otimizador
# ---------------------------
model = BertForPreTraining.from_pretrained("bert-base-uncased").to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# ---------------------------
# 7) Loop de treino curto (didático)
# ---------------------------
EPOCHS = 1
model.train()
for epoch in range(1, EPOCHS+1):
    running = 0.0
    for step, batch in enumerate(train_loader, 1):
        batch = {k: v.to(device) for k,v in batch.items()}
        optimizer.zero_grad()
        out = model(**batch)  # loss = MLM + NSP
        loss = out.loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        running += loss.item()

        if step % 20 == 0:
            print(f"[epoch {epoch} step {step:03d}] loss={running/20:.4f}")
            running = 0.0

# ---------------------------
# 8) Funções de inspeção: MLM top-k e NSP probs
# ---------------------------
@torch.no_grad()
def inspect_mlm(tokenizer, model, a, b, topk=5):
    model.eval()
    enc = encode_pair(a, b)
    # força UM [MASK] manualmente para demonstrar (na primeira palavra de B se possível)
    ids = enc["input_ids"].clone()
    # procurar um token "previstável" em B (token_type_id==1) que não seja especial
    ttypes = enc["token_type_ids"]
    chosen_idx = None
    for i in range(ids.size(0)):
        if (ttypes[i] == 1) and (ids[i] not in {CLS_ID, SEP_ID, PAD_ID}):
            chosen_idx = i; break
    if chosen_idx is None:
        # fallback: primeiro token não-especial
        for i in range(ids.size(0)):
            if ids[i] not in {CLS_ID, SEP_ID, PAD_ID}:
                chosen_idx = i; break
    original = ids[chosen_idx].item()
    ids[chosen_idx] = MASK_ID

    batch = {
        "input_ids": ids.unsqueeze(0).to(device),
        "token_type_ids": enc["token_type_ids"].unsqueeze(0).to(device),
        "attention_mask": enc["attention_mask"].unsqueeze(0).to(device),
    }
    out = model(**batch)
    logits = out.prediction_logits[0, chosen_idx]  # (V,)
    probs = torch.softmax(logits, dim=-1)
    vals, idxs = torch.topk(probs, k=topk)
    toks = [tokenizer.decode([i]) for i in idxs.tolist()]
    return {
        "masked_position": chosen_idx,
        "original_token": tokenizer.decode([original]),
        "topk": list(zip(toks, [float(v) for v in vals]))
    }

@torch.no_grad()
def inspect_nsp(tokenizer, model, a, b):
    model.eval()
    enc = encode_pair(a, b)
    batch = {
        "input_ids": enc["input_ids"].unsqueeze(0).to(device),
        "token_type_ids": enc["token_type_ids"].unsqueeze(0).to(device),
        "attention_mask": enc["attention_mask"].unsqueeze(0).to(device),
    }
    out = model(**batch)
    # out.seq_relationship_logits: (B, 2) -> [IsNext, NotNext]
    logits = out.seq_relationship_logits[0]
    probs = torch.softmax(logits, dim=-1)
    return {"IsNext": float(probs[0]), "NotNext": float(probs[1])}

# ---------------------------
# 9) Testes: ver predições após o treino curto
# ---------------------------
tests = [
    ("the cat sat on the mat.", "it [MASK] out the window."),
    ("paris is the capital of france.", "the eiffel tower is in paris."),
    ("deep learning changed natural language processing.", "bert learns bidirectional context."),
    ("paris is the capital of france.", "masked language modeling is powerful."),  # NotNext provável
]

print("\n=== INSPEÇÃO DE MLM (top-5 para um [MASK]) ===")
for a, b in tests:
    r = inspect_mlm(tokenizer, model, a, b, topk=5)
    print(f"\nA: {a}\nB: {b}")
    print(f"  posição mascarada: {r['masked_position']}")
    print(f"  token original (se disponível): {r['original_token']!r}")
    for tok, p in r["topk"]:
        print(f"   -> {tok:<12} p={p:.4f}")

print("\n=== INSPEÇÃO DE NSP (probabilidades) ===")
for a, b in tests:
    p = inspect_nsp(tokenizer, model, a, b)
    print(f"\nA: {a}\nB: {b}")
    print(f"  P(IsNext)={p['IsNext']:.3f}  |  P(NotNext)={p['NotNext']:.3f}")

# 5.5 — Fine-Tuning do BERT e o papel do token `[CLS]`

<br/>
<img src="https://raw.githubusercontent.com/bmnogueira-ufms/TopicosIA-2025-02/main/images/bert_ft_tasks.png" width="50%%">

Fonte: Devlin et al. (2018)

Após o pré-treinamento, o BERT é uma poderosa **base de conhecimento linguístico geral**, mas ainda **não sabe realizar tarefas específicas** (como classificar sentimentos ou responder perguntas).  
O **fine-tuning** é o processo que adapta esse conhecimento para **tarefas supervisionadas específicas**.

---

## Conceito de Fine-Tuning

O *fine-tuning* consiste em:

1. **Reaproveitar os pesos do modelo pré-treinado**, que já aprenderam relações semânticas e sintáticas da linguagem.  
2. **Acrescentar uma pequena camada de saída** (geralmente linear) adaptada à tarefa desejada.  
3. **Treinar o modelo completo novamente**, mas em um **novo conjunto de dados rotulado**, com uma **taxa de aprendizado menor** — para ajustar suavemente os pesos sem “destruir” o conhecimento linguístico aprendido.

Essa abordagem faz parte do paradigma de **transfer learning**:
- o pré-treinamento fornece **conhecimento geral** sobre linguagem;
- o fine-tuning **especializa** esse conhecimento para um contexto ou tarefa.

---

## Como o Fine-Tuning é Feito

O processo segue o mesmo pipeline básico:

> Texto → Tokenização → Embeddings → Camadas do BERT → Saída Adaptada → Função de Perda

Dependendo da tarefa, usamos diferentes **estruturas de saída**:

| Tipo de Tarefa | Saída Esperada | Cabeça Adicional | Exemplo |
|----------------|----------------|------------------|----------|
| Classificação de Sentença | 1 vetor (logits) | Linear + Softmax | Análise de Sentimento |
| Classificação de Token | 1 vetor por token | Linear + Softmax | NER, POS Tagging |
| Perguntas e Respostas | Vetores de início e fim | Linear (dupla saída) | SQuAD |
| Similaridade entre sentenças | 2 embeddings | Camada de similaridade (cosine) | STS |

---

## O Papel do Token `[CLS]`

O token especial `[CLS]` (*classification token*) é adicionado **no início de toda sequência**.  
Durante o pré-treinamento, ele é associado à tarefa **Next Sentence Prediction (NSP)**, ou seja, o modelo já aprendeu a representar **o significado global da sentença ou do par de sentenças** nele.

Por isso, no *fine-tuning* para tarefas de **classificação**, usamos **somente o vetor final de `[CLS]`** como representação da entrada completa.

### Exemplo:

> Entrada: [CLS] O filme foi excelente ! [SEP]

O BERT processa todos os tokens por 12 camadas de atenção.  
Ao final, o vetor correspondente a `[CLS]` contém uma **representação contextual global** do significado da frase.

> Esse vetor é passado para uma **camada linear de classificação** que gera os logits finais (ex: positivo / negativo).

Matematicamente:

$$
\text{logits} = W \cdot h_{[CLS]} + b
$$

onde:
- $h_{[CLS]}$ é o embedding final do token `[CLS]`;  
- $W$ e $b$ são parâmetros da nova camada linear.

---

## Fine-Tuning em Diferentes Tarefas

### (a) Classificação de Sentenças
- Entrada: `[CLS] sentença [SEP]`
- Saída: vetor `[CLS]` → camada linear → softmax.

### (b) Classificação de Tokens (ex: NER)
- Entrada: `[CLS] frase [SEP]`
- Saída: vetor contextual **de cada token**, passando cada um por uma camada linear.

### (c) Perguntas e Respostas
- Entrada: `[CLS] pergunta [SEP] contexto [SEP]`
- Saída: duas camadas lineares preveem índices de início e fim no contexto.

---

## Treinamento Suave (Small Learning Rate)

Durante o *fine-tuning*, usamos:
- **taxas de aprendizado pequenas (ex: 2e-5 a 5e-5)**;
- **poucas épocas (2–4)**;
- **batch sizes pequenos (16–32)**;
- **métodos de regularização leves** (como dropout).

A ideia é **ajustar levemente** o modelo, mantendo a “gramática geral” da linguagem que o BERT já aprendeu.

---

## Visualização Conceitual

<pre>
[Texto de Entrada] → [Tokenização]
          ↓
     [CLS]  token1  token2  ...  [SEP]
          ↓
     Embeddings + Posições
          ↓
      Camadas BERT
          ↓
      Vetor [CLS]  ← representa a sentença
          ↓
   Camada Linear (Nova)
          ↓
   Saída final (ex: sentimento positivo)
</pre>

O BERT já aprendeu “como o idioma funciona”; o fine-tuning ensina “como resolver o seu problema específico”.
    
- O token [CLS] é o resumo semântico da sentença — ele condensa, em um vetor, as informações que fluíram por toda a rede de atenção.
    
- Essa adaptação é tão eficiente que, muitas vezes, basta menos de 1 hora de treino para atingir desempenho de ponta em diversas tarefas.

## Como o `[CLS]` é atualizado durante o Fine-Tuning

O token especial `[CLS]` é adicionado **no início da sequência** de entrada:

> [CLS] O filme foi ótimo [SEP]

Durante o **forward pass** do BERT:

1. Ele recebe um embedding inicial aprendido (como qualquer palavra).
2. Passa por todas as camadas do Transformer — o `[CLS]` “observa” todas as outras palavras via *self-attention*.
3. O vetor final do `[CLS]` (geralmente de 768 dimensões) é interpretado como o **resumo contextual da sentença**.
4. Esse vetor é então enviado a uma camada linear + *softmax* para produzir as probabilidades das classes:

$$
\hat{y} = \text{Softmax}(W_{cls} \cdot h_{cls} + b)
$$

onde \( h_{cls} \) é o embedding final do `[CLS]`.

---

### A Função de Custo e o Gradiente

Para uma tarefa de classificação binária, usamos a **entropia cruzada**:

$$
\mathcal{L} = - \sum_i y_i \log(\hat{y}_i)
$$

Durante o **backpropagation**, o gradiente da perda em relação a \( h_{cls} \) é:

$$
\frac{\partial \mathcal{L}}{\partial h_{cls}} = W_{cls}^\top (\hat{y} - y)
$$

Esse gradiente é propagado de volta, atualizando:
- o vetor `[CLS]`;
- as projeções da atenção e camadas feedforward;
- e até mesmo os embeddings iniciais.

Assim, o `[CLS]` aprende a **codificar padrões discriminativos** úteis para a tarefa.

---

### Intuição

Imagine duas frases:

| Sentença | Rótulo |
|-----------|---------|
| “O filme foi maravilhoso!” | 1 (positivo) |
| “O filme foi péssimo.” | 0 (negativo) |

Inicialmente, os vetores `[CLS]` dessas frases são aleatórios ou neutros.  
Após algumas iterações:

- O `[CLS]` das frases **positivas** é “puxado” para uma região do espaço vetorial associada a altas probabilidades de classe 1.
- O `[CLS]` das frases **negativas** é empurrado para outra região, favorecendo a classe 0.

O espaço vetorial se organiza assim:

Espaço vetorial após fine-tuning

<pre>
↑ Classe 1 (positivo)
│
│      o    o   o
│        o
│
│
│
│     x
│  x     x
└────────────────────→ Classe 0 (negativo)
</pre>


---

### Exemplo Numérico Simplificado

O exemplo abaixo mostra **como o gradiente atua no vetor `[CLS]`**.

- h_cls = torch.tensor([0.2, 0.1, 0.4])   # embedding [CLS]
- W_cls = torch.tensor([[ 0.5, -0.3, 0.1],
- [-0.2,  0.4, 0.6]])   # pesos do classificador
- b = torch.tensor([0.0, 0.0])

A saída antes do ajuste pode ser algo como:

> tensor([0.63, 0.37])   # predição: classe 0

Se o rótulo verdadeiro é 1, a perda:

$$
L = -\log(0.37)
$$

gera um gradiente que:

- diminui a confiança na classe 0;
- aumenta as direções em $ h_{cls} $ que favorecem a classe 1.

Após algumas iterações, o vetor `[CLS]` é ajustado de modo que:

$ Softmax(W_cls ⋅ h_cls) ≈ [0.1, 0.9] $

---

### Interpretação Final

Depois do *fine-tuning*:
- O vetor `[CLS]` é um **resumo contextual** ajustado à tarefa.
- A camada linear \( W_{cls} \) aprende um **hiperplano de decisão** que separa `[CLS]` por classe.
- Por isso, podemos usar apenas os vetores `[CLS]` (sem o resto do BERT) para visualizar como o modelo organiza o espaço semântico — o que faremos a seguir com **t-SNE/UMAP**.

In [ ]:
import torch
import torch.nn.functional as F

# Vetor [CLS] e pesos iniciais (3 dimensões → 2 classes)
h_cls = torch.tensor([0.2, 0.1, 0.4], requires_grad=True)
W_cls = torch.tensor([[ 0.5, -0.3, 0.1],
                      [-0.2,  0.4, 0.6]], requires_grad=True)
b = torch.tensor([0.0, 0.0], requires_grad=True)

# Forward pass
logits = W_cls @ h_cls + b
probs = F.softmax(logits, dim=0)
print("Probabilidades iniciais:", probs.detach().numpy())

# Rótulo verdadeiro: classe 1 (positiva)
target = torch.tensor([0., 1.])

# Perda (entropia cruzada)
loss = -torch.sum(target * torch.log(probs))
print("Perda inicial:", loss.item())

# Backprop
loss.backward()

print("\nGradiente em relação a h_cls:", h_cls.grad)
print("Gradiente em relação a W_cls:", W_cls.grad)

# Atualiza manualmente (1 passo de SGD)
lr = 0.5
with torch.no_grad():
    h_cls -= lr * h_cls.grad
    W_cls -= lr * W_cls.grad

logits_new = W_cls @ h_cls + b
probs_new = F.softmax(logits_new, dim=0)
print("\nProbabilidades após atualização:", probs_new.detach().numpy())

## 5.6 — Fine-tuning do BERT na prática (Classificação de Sentenças)

Nesta seção, vamos **ajustar** (fine-tune) um BERT pré-treinado para uma tarefa supervisionada de **classificação de sentenças** (sentimento).  
Usaremos o modelo `BertForSequenceClassification`, que adiciona automaticamente uma **cabeça linear** sobre o vetor do **token `[CLS]`** (na prática, o "pooled output" = `tanh(W * h_[CLS] + b)`).

### Pipeline

1. **Carregar dados** (GLUE/SST-2; se indisponível, usamos um fallback em memória).
2. **Tokenizar** com `BertTokenizerFast` (`[CLS] ... [SEP]` automaticamente).
3. **Instanciar o modelo** `BertForSequenceClassification(num_labels=2)`.
4. **Treinar** com `Trainer` (taxa pequena, poucas épocas).
5. **Avaliar** (accuracy, F1) e **inspecionar previsões**.

> Intuição: durante o fine-tuning, o vetor contextual do `[CLS]` passa por uma camada linear + softmax.  
> A perda supervisionada **empurra** o embedding do `[CLS]` para organizar o espaço semântico de modo a separar as classes (positivo/negativo).

In [ ]:
# ============================================================
# BERT Fine-tuning — Classificação (SST-2 com fallback) + SANEAMENTO ULTRARROBUSTO
# ============================================================
import os, random, math, inspect
import numpy as np
import torch

# ---------------------------
# 0) Setup determinístico
# ---------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ---------------------------
# 1) (Colab) instalar libs
# ---------------------------
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install -U "transformers>=4.39" "datasets>=2.14" "accelerate>=0.28" "evaluate>=0.4"

# ---------------------------
# 2) Imports principais
# ---------------------------
# import transformers
from transformers import (
    BertTokenizerFast, BertForSequenceClassification,
    Trainer, TrainingArguments
)
print("transformers:", transformers.__version__)

# Helper: TrainingArguments compatível
def build_training_arguments(**kwargs) -> TrainingArguments:
    sig = inspect.signature(TrainingArguments.__init__)
    allowed = set(sig.parameters.keys()); allowed.discard("self")
    filtered = {k: v for k, v in kwargs.items() if k in allowed}
    dropped = [k for k in kwargs if k not in allowed]
    if dropped:
        print("[Aviso] parâmetros ignorados:", dropped)
    return TrainingArguments(**filtered)

# ---------------------------
# 3) Dataset: GLUE/SST-2 → fallback
# ---------------------------
USE_FALLBACK = False
train_texts, train_labels = [], []
val_texts,   val_labels   = [], []

try:
    from datasets import load_dataset
    glue = load_dataset("glue", "sst2")
    train_texts = glue["train"]["sentence"]
    train_labels= glue["train"]["label"]
    val_texts   = glue["validation"]["sentence"]
    val_labels  = glue["validation"]["label"]
    print(f"GLUE/SST-2 carregado: train={len(train_texts)}  val={len(val_texts)}")
except Exception as e:
    print("[AVISO] Falha ao carregar GLUE/SST-2:", repr(e))
    USE_FALLBACK = True
    train_pairs = [
        ("i loved the movie, it was fantastic and touching.", 1),
        ("what a terrible waste of time.", 0),
        ("an excellent script and strong performances.", 1),
        ("the film is boring and predictable.", 0),
        ("absolutely wonderful and inspiring!", 1),
        ("bad acting and poor dialogue.", 0),
        ("smart, funny, and well paced.", 1),
        ("i hated every minute of it.", 0),
        ("a delightful surprise with great music.", 1),
        ("the plot makes no sense at all.", 0),
        ("remarkably good for a low budget.", 1),
        ("painfully slow and unoriginal.", 0),
    ]
    random.shuffle(train_pairs)
    train_texts = [t for t,_ in train_pairs]
    train_labels= [y for _,y in train_pairs]
    val_pairs = [
        ("wonderful direction and amazing visuals.", 1),
        ("awful script and worse execution.", 0),
        ("i really enjoyed this film.", 1),
        ("it was not enjoyable at all.", 0),
    ]
    val_texts  = [t for t,_ in val_pairs]
    val_labels = [y for _,y in val_pairs]
    print(f"Fallback: train={len(train_texts)}  val={len(val_texts)}")

# ---------------------------
# 4) SANEAMENTO ULTRARROBUSTO
# ---------------------------
from collections.abc import Iterable

def _is_nan(x):
    try:
        return bool(np.isnan(x))  # cobre floats numpy e python
    except Exception:
        return False

def _to_plain_str(x):
    # bytes → str
    if isinstance(x, (bytes, bytearray)):
        try:
            x = x.decode("utf-8", "ignore")
        except Exception:
            x = str(x)
    # numpy types / outros
    if isinstance(x, (np.generic,)):
        x = x.item()
    # dict comum com chave 'text' ou 'sentence'
    if isinstance(x, dict):
        for k in ("text", "sentence", "content"):
            if k in x and isinstance(x[k], (str, bytes, bytearray)):
                return _to_plain_str(x[k])
        # último recurso: string da representação
        return str(x)
    # número → str
    if isinstance(x, (int, float, np.integer, np.floating)) and not _is_nan(x):
        return str(x)
    # strings “normais”
    if isinstance(x, str):
        return x
    # iteráveis de strings → junta
    if isinstance(x, Iterable) and not isinstance(x, (str, bytes, bytearray)):
        # achata e junta com espaço
        flat = []
        for item in x:
            s = _to_plain_str(item)
            if s is not None and s.strip():
                flat.append(s.strip())
        return " ".join(flat) if flat else None
    # fallback
    try:
        return str(x)
    except Exception:
        return None

def clean_xy(texts, labels, max_bad_print=5, name="train"):
    X, y = [], []
    bad_samples = []
    # Se vier numpy array / pandas Series, transforma
    if hasattr(texts, "tolist"):
        texts = texts.tolist()
    if hasattr(labels, "tolist"):
        labels = labels.tolist()

    for t, l in zip(texts, labels):
        # remover None/NaN
        if t is None:
            bad_samples.append(("None", t)); continue
        if _is_nan(t):
            bad_samples.append(("NaN", t)); continue

        s = _to_plain_str(t)
        if s is None:
            bad_samples.append(("unconvertible", t)); continue
        s = s.strip()
        if not s:
            bad_samples.append(("empty after strip", t)); continue

        try:
            lbl = int(l)
        except Exception:
            # alguns datasets retornam np.int64 etc.
            try:
                lbl = int(np.array(l).item())
            except Exception:
                bad_samples.append(("bad label", (t, l))); continue

        X.append(s); y.append(lbl)

    if bad_samples:
        print(f"[{name}] {len(bad_samples)} amostra(s) removida(s) por formato inválido.")
        for i, (reason, sample) in enumerate(bad_samples[:max_bad_print], 1):
            print(f"  #{i} motivo={reason}  exemplo={repr(sample)[:120]}")

    return X, y

train_texts, train_labels = clean_xy(train_texts, train_labels, name="train")
val_texts,   val_labels   = clean_xy(val_texts,   val_labels,   name="val")

print(f"Textos saneados: train={len(train_texts)}  val={len(val_texts)}")
assert len(train_texts) == len(train_labels) and len(val_texts) == len(val_labels)

# checagem final (se ainda falhar, imprime tipos “estranhos”)
def _debug_types(name, xs, n=5):
    weird = [(i, type(x).__name__, repr(x)[:80]) for i, x in enumerate(xs) if not isinstance(x, str)]
    if weird:
        print(f"[DEBUG] {name}: itens não-string após saneamento:", weird[:n])

_debug_types("train_texts", train_texts)
_debug_types("val_texts",   val_texts)

# ---------------------------
# 5) Tokenizer e tokenização
# ---------------------------
MODEL_NAME = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
MAX_LEN = 128

def tokenize_batch(texts):
    # segurança extra: garante list[str]
    assert isinstance(texts, (list, tuple)), "tokenize_batch espera list/tuple de strings."
    assert all(isinstance(t, str) for t in texts), "tokenize_batch recebeu itens não-string."
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

train_enc = tokenize_batch(train_texts)
val_enc   = tokenize_batch(val_texts)

# ---------------------------
# 6) Dataset PyTorch
# ---------------------------
class TorchTextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels    = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_ds = TorchTextDataset(train_enc, train_labels)
val_ds   = TorchTextDataset(val_enc,   val_labels)

# ---------------------------
# 7) Modelo (usa pooled_output = [CLS])
# ---------------------------
num_labels = 2
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels).to(device)

# ---------------------------
# 8) Métricas
# ---------------------------
try:
    import evaluate
    acc_metric = evaluate.load("accuracy")
    f1_metric  = evaluate.load("f1")
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        r1 = acc_metric.compute(predictions=preds, references=labels)
        r2 = f1_metric.compute(predictions=preds, references=labels, average="weighted")
        return {"accuracy": r1["accuracy"], "f1": r2["f1"]}
except Exception:
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = (preds == labels).mean()
        return {"accuracy": float(acc)}

# ---------------------------
# 9) Treinamento com Trainer
# ---------------------------
EPOCHS = 2 if not USE_FALLBACK else 4
BATCH  = 16 if not USE_FALLBACK else 8

args = build_training_arguments(
    output_dir="bert-ft-sst2",
    evaluation_strategy="epoch",
    save_strategy="no",
    report_to="none",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    num_train_epochs=EPOCHS,
    fp16=torch.cuda.is_available(),
    seed=SEED,
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

print("\n=== Iniciando fine-tuning do BERT ===")
train_out = trainer.train()
eval_out  = trainer.evaluate()
print("\nResultados de validação:", eval_out)

# ---------------------------
# 10) Teste prático — previsões e inspeção
# ---------------------------
def tokenize_one(text):
    return tokenizer(
        text, padding="max_length", truncation=True, max_length=MAX_LEN, return_tensors="pt"
    )

def predict(texts):
    model.eval()
    assert isinstance(texts, (list, tuple)), "predict espera list/tuple de strings."
    enc = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model(**enc)
        probs = torch.softmax(out.logits, dim=-1).cpu().numpy()
        preds = probs.argmax(axis=-1)
    return preds, probs

samples = [
    "i absolutely loved this movie!",
    "this was a complete waste of time.",
    "the performances are strong and convincing.",
    "the plot is weak and the pacing is terrible.",
]

preds, probs = predict(samples)
label_names = ["negative", "positive"]
print("\n=== Amostras de previsão ===")
for s, p, pr in zip(samples, preds, probs):
    print(f"- {s}\n  -> pred: {label_names[p]}  probs={pr}")

In [ ]:
# Para inspecionar manualmente o vetor [CLS] (pooled_output):
# with torch.no_grad():
#     enc = tokenize_one("a great movie.").to(device)
#     outputs = model.bert(**enc, return_dict=True)
#     pooled = outputs.pooler_output
#     print("pooled_output shape:", pooled.shape)

# 5.7 BERT em tarefas de *Question Answering*

No *Question Answering* (QA) — como em *SQuAD (Stanford Question Answering Dataset)* —  
o modelo deve **encontrar o trecho exato da resposta** dentro de um texto dado.

---

## Estrutura de Entrada

O input do BERT é a **concatenação** da *pergunta* e do *contexto*:

```text
[CLS] Quem descobriu o Brasil? [SEP] Pedro Álvares Cabral descobriu o Brasil em 1500. [SEP]
```

- `[CLS]` → marcador de início (como sempre)  
- `[SEP]` → separa a pergunta do contexto  
- cada token (pergunta + contexto) recebe *token embeddings*, *segment embeddings* (A = pergunta, B = contexto) e *position embeddings*.

---

## O que o BERT aprende a prever

O modelo de QA **não gera texto**.  
Ele **prediz dois índices** dentro da sequência de tokens do contexto:

1. **Start token** → onde a resposta começa.  
2. **End token** → onde a resposta termina.

Exemplo:

```text
Tokens: [CLS] Quem descobriu o Brasil ? [SEP] Pedro Álvares Cabral descobriu o Brasil em 1500 . [SEP]
Index:   0     1     2          3      4   5    6      7       8          9  10    11   12  13   14

Resposta correta: "Pedro Álvares Cabral"
→ start = 5, end = 8
```

---

## Saídas do modelo

O BERT retorna **um vetor por token** (como sempre).  
Para QA, adicionamos **duas camadas lineares** sobre esses vetores:

$begin:math:display$
\\begin{align}
\\text{StartLogits} &= W_{start} \\cdot H + b_{start} \\\\
\\text{EndLogits}   &= W_{end}   \\cdot H + b_{end}
\\end{align}
$end:math:display$

onde $begin:math:text$ H $end:math:text$ é a matriz $begin:math:text$(n_{tokens}, d_{model})$end:math:text$ com as representações finais.

- `start_logits`: pontua cada token como *início possível da resposta*  
- `end_logits`: pontua cada token como *fim possível da resposta*  

O modelo escolhe o par `(start, end)` com a soma dos logits mais alta.

---

## Exemplo conceitual

Suponha que a saída de *start_logits* e *end_logits* seja:

| Token | Start logit | End logit |
|:------|-------------:|----------:|
| `[CLS]` | -5.3 | -5.0 |
| `Pedro` | **7.2** | 0.5 |
| `Álvares` | 5.8 | 6.1 |
| `Cabral` | 0.8 | **7.5** |
| `descobriu` | -0.4 | -2.1 |

O melhor par `(start=Pedro, end=Cabral)` forma a resposta:
```
Pedro Álvares Cabral
```

---

## Função de custo

Durante o treino, temos rótulos reais `start_true` e `end_true`.

A perda é a soma das entropias cruzadas independentes:

$begin:math:display$
\\mathcal{L} = CE(\\hat{y}_{start}, y_{start}) + CE(\\hat{y}_{end}, y_{end})
$end:math:display$

Essa perda ajusta todos os vetores de saída (não só o `[CLS]`),  
fazendo com que as representações do início e do fim da resposta se tornem mais distintas.

---

## Interpretação das saídas

No final:

- cada token do contexto tem uma pontuação de “probabilidade de início/fim”;
- o `[CLS]` continua existindo (geralmente usado para *no-answer* em datasets como SQuAD 2.0);
- o modelo **não gera texto**, apenas **aponta spans** dentro do contexto original.
- o `[CLS]` é usado apenas como referência global (ou para no-answer),
- o foco está nas representações **token a token** do contexto,
- a saída são **dois vetores de logits** (start e end), que identificam o *span* da resposta.

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForQuestionAnswering

# 1) Modelo já fine-tuned em SQuAD
MODEL_NAME = "bert-large-uncased-whole-word-masking-finetuned-squad"
# Alternativa mais leve:
# MODEL_NAME = "distilbert-base-uncased-distilled-squad"

tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
model = BertForQuestionAnswering.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# 2) Exemplo de pergunta e contexto
question = "Who discovered Brazil?"
context = (
    "Pedro Álvares Cabral discovered Brazil in the year 1500 during a Portuguese expedition. "
    "He is considered one of the key figures in the Age of Discovery."
)

# 3) Tokenização com offsets (para mapear tokens no texto original)
enc = tokenizer(
    question,
    context,
    return_tensors="pt",
    truncation=True,
    max_length=384,
    return_offsets_mapping=True
)
enc = {k: v.to(device) for k, v in enc.items()}
offsets = enc["offset_mapping"][0].tolist()
ttypes = enc["token_type_ids"][0].tolist()

# 4) Forward
with torch.no_grad():
    outputs = model(**{k: v for k, v in enc.items() if k != "offset_mapping"})
start_logits, end_logits = outputs.start_logits[0], outputs.end_logits[0]

# 5) Filtrar apenas tokens do contexto (segment=1)
context_mask = torch.tensor([1 if t == 1 else 0 for t in ttypes], device=device, dtype=torch.bool)
start_logits = start_logits.masked_fill(~context_mask, -1e9)
end_logits = end_logits.masked_fill(~context_mask, -1e9)

# 6) Selecionar melhor par (start, end)
best_start = torch.argmax(start_logits).item()
best_end = torch.argmax(end_logits).item()
if best_end < best_start:
    best_end = best_start

# 7) Reconstruir texto a partir dos offsets
start_char, _ = offsets[best_start]
_, end_char = offsets[best_end]
answer = context[start_char:end_char]

print("Pergunta:", question)
print("Melhor resposta:", repr(answer))
print(f"(start={best_start}, end={best_end})")

# 8) Mostrar top-k spans
def topk_spans(start_logits, end_logits, offsets, context, k=5, max_len=30):
    s, e = start_logits.cpu().numpy(), end_logits.cpu().numpy()
    top_s, top_e = np.argsort(s)[::-1][:k*5], np.argsort(e)[::-1][:k*5]
    spans = []
    for i in top_s:
        for j in top_e:
            if j < i or (j - i + 1) > max_len:
                continue
            score = s[i] + e[j]
            sc, ec = offsets[i][0], offsets[j][1]
            spans.append((score, context[sc:ec]))
    spans.sort(key=lambda x: x[0], reverse=True)
    return spans[:k]

print("\nTop-5 spans mais prováveis:")
for score, span in topk_spans(start_logits, end_logits, offsets, context):
    print(f"score={score:6.2f} | {span}")